In [1]:
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel

c:\Users\SBAUser\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [4]:
sent = '근육이 커지기 위해서는'

In [5]:
input_ids = tokenizer.encode(sent, return_tensors='pt')
print(input_ids)

tensor([[33245, 10114, 12748, 11357]])


In [7]:
output = model.generate(input_ids, max_length = 128, repetition_penalty=2.0, use_cache = True)
# repetition_penalty : 반복시 패널티 부과 > 생성 결과에서 반복되는 단어나 구 제거하기 위함
# use_cache : 이전 연산 결과 캐시 저장 >> 재사용
output_ids = output.numpy().tolist()[0]
print(output_ids)


[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


In [8]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

In [9]:
# numpy 로 top5 추출

import numpy as np
import torch

In [10]:
output = model(input_ids)

In [11]:
output.logits
# 각 단어에 대한 점수를 나타내는 값
# >> softmax 함수 통과 후, 확률값으로 변환
# >> argmax(가장 가능성이 높은 확률값) 활용
# >> 다음 단어 예측

tensor([[[-5.2968, -5.2453, -4.7947,  ...,  0.0246, -1.1457, -0.6493],
         [-4.7908, -5.4113, -4.8760,  ..., -0.9310, -3.6518, -5.8186],
         [-4.0486, -3.5685, -5.7360,  ..., -2.4099, -2.4636, -3.5292],
         [-5.4558, -6.0249, -6.1399,  ..., -0.2245, -4.0262, -3.2469]]],
       grad_fn=<UnsafeViewBackward0>)

In [12]:
output.logits.shape

torch.Size([1, 4, 51200])

In [13]:
output.logits[0,-1]

tensor([-5.4558, -6.0249, -6.1399,  ..., -0.2245, -4.0262, -3.2469],
       grad_fn=<SelectBackward0>)

In [14]:
logits = output.logits[0,-1]
logits.shape
# torch.Szie([51200])
# >> 51200개의 vocab_size

torch.Size([51200])

In [15]:
top5 = torch.topk(logits, k=5)

# 전체 vocab_size 에서 가장 확률이 높은 5개 추출

tokens = [tokenizer.decode(token_id) for token_id in top5.indices.tolist()]

In [16]:
print(tokens)

['무엇보다', '우선', '반드시', '피부', '무엇보다도']


In [17]:
# Numpy 활용, top5로 문장 생성하기
import random

In [19]:
while len(input_ids[0]) < 50 :
    with torch.no_grad() :
        output = model(input_ids)
    logits = output.logits[0,-1]
    top5 = torch.topk(logits, k = 5)
    token_id = random.choice(top5.indices.tolist())
    input_ids = torch.cat([input_ids, torch.tensor([[token_id]])], dim=1)

# top5중 random choice(무작위 추출, 선택)
# >> 기존 입력(input_ids) + top5 합친 후 >> 다시 생성

In [20]:
print(input_ids)

tensor([[33245, 10114, 12748, 11357, 11488, 12443,  7824, 15064,  8137,  9277,
          9030, 12443, 31208, 13023, 12209, 10161,  9932, 13525, 22289, 10386,
         12923,  9330, 11187, 14931,  7673, 11356, 28037, 10826,  7335,  8704,
          9346, 17049, 35838, 10599, 15177,  8368,  8243,  9244, 11356,  9207,
          7791, 23879, 39306, 13579, 13023, 11584, 18595, 29729, 17764,  7162]])


In [21]:
len(input_ids[0])

50

In [22]:
tokenizer.decode(input_ids[0])

'근육이 커지기 위해서는 반드시 영양소 보충을 하는 등 영양관리를 해야 한다고 전문가는 말했다.\n영양소가 풍부한 음식은 소화불량을 줄여준다.\n또한 체중을 줄이는 것도 중요하다.\n체중 감량을 위해선 무엇보다 규칙적인 운동을 해야 하며 충분한 수분 섭취는'